In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d dorianlazar/medium-articles-dataset

In [ ]:
! mkdir dataset

In [ ]:
!unzip /content/medium-articles-dataset.zip -d dataset

import os
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

In [ ]:
medium_data = pd.read_csv('/content/dataset/medium_data.csv')
medium_data.head()

In [ ]:
print("Number of records: ", medium_data.shape[0])
print("Number of fields: ", medium_data.shape[1])

In [ ]:
medium_data['title']

In [ ]:
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'\xa0',u' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace('\u200a',' '))     


In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((total_words, embedding_dim))
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        word, *vector = line.split()
        if word in tokenizer.word_index:
            idx = tokenizer.word_index[word]
            embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

In [ ]:
from keras.initializers import Constant
from tensorflow.keras.layers import LSTM, Embedding
# Define the maximum sequence length
max_sequence_length = 39  # You should replace this with your actual maximum sequence length

# Create the model without setting the weights initially
model = tf.keras.Sequential([
    Embedding(input_dim=total_words, output_dim=embedding_dim,embeddings_initializer=Constant(embedding_matrix),trainable=False, input_shape=(max_sequence_length,)),
    LSTM(1000, return_sequences=True),
    LSTM(1000),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])


# Compile the model
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

# Print model summary to check the number of weights expected by the Embedding layer
print(model.summary())

In [ ]:
history = model.fit(x, y, epochs=50, verbose=1)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
text = "My favourite book"
next_words = 20
  
for i in range(next_words):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predict_x=model.predict(token_list) 
    predicted=np.argmax(predict_x,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    text += " " + output_word
print(text)

In [ ]:
pickle.dump(model,open("model.pkl","wb"))